In [117]:
import numpy as np
import math

def sigmoid(x):
  return 1 / (1 + math.exp(-x))

In [118]:
class Neuron:
    def __init__(self):
        self.output = 0
    def activate(self, inputs: np.ndarray, weights: np.ndarray):
        logit = np.dot(inputs, weights)
        self.output = sigmoid(logit)
        return self.output

In [119]:
class Layer:
    def __init__(self, size):
        self.neurons = [Neuron() for i in range(size)]
        self.y = np.zeros(size)
        self.size = size
    def propagate(self, inputs: np.ndarray, weights: np.ndarray):
        # inputs: vector of values (x)
        # weights: matrix where weights[i][j] is the weight going into i-th neuron from j-th input
        for i in range(len(self.neurons)):
            neuron = self.neurons[i]
            # compute output for each neuron
            self.y[i] = neuron.activate(inputs, weights[i])
        return self.y

In [120]:
class Network:
    def __init__(self, input_size, output_size):
        # weights is an array of weight matricies
        self.weights = []
        self.input_size = input_size
        self.output_size = output_size
        
        input_layer = Layer(input_size)
        output_layer = Layer(output_size)
        self.layers = [input_layer, output_layer]
        
    def add_layer(self, layer: Layer):
        self.layers = self.layers[:-1] + [layer] + [self.layers[-1]]
        
    def init_weights(self):
        # each input goes straight to its dedicated neuron in the input layer
        self.weights = [np.identity(self.input_size)]
        
        for i in range(len(self.layers) - 1):
            this_layer = self.layers[i]
            next_layer = self.layers[i+1]
            self.weights.append(np.random.rand(next_layer.size, this_layer.size))
            
    def propagate(self, inputs: np.ndarray):
        i = 0
        y = inputs
        while i < len(self.layers):
            current_layer = self.layers[i]
            y = current_layer.propagate(y, self.weights[i])
            i += 1
        return y

In [121]:
mynet.layers[0].neurons

[<__main__.Neuron at 0x7f502e112b50>, <__main__.Neuron at 0x7f502e112700>]

In [166]:
mynet = Network(3, 1)
mynet.add_layer(Layer(2))
mynet.init_weights()
mynet.propagate(np.random.rand(3))

array([0.6672005])

In [168]:
mynet.weights

[array([[1., 0., 0.],
        [0., 1., 0.],
        [0., 0., 1.]]),
 array([[0.30794211, 0.19676281, 0.54494325],
        [0.53768707, 0.59337697, 0.7256727 ]]),
 array([[0.88982396, 0.16996276]])]

In [125]:
mynet.weights
            

[array([[1., 0.],
        [0., 1.]]),
 array([[0.30090808, 0.67801843]])]

array([0.66117293])

In [28]:
inputs = np.array([10, 20])
weights = np.array([
    [0, 1], 
    [1, 0], 
    [1, 1]
])

layer.propagate(inputs, weights)

array([20., 10., 30.])

In [39]:
test[0][0] = np.array([1,2,3])

ValueError: setting an array element with a sequence.

In [44]:
a = [1,2,3]
a.concat([4,5,6])
a

AttributeError: 'list' object has no attribute 'concat'

In [16]:
v1 = np.array([1,2,3])
v2 = np.array([1,0,1])
myneuron = Neuron()
myneuron.activate(v1, v2)

4

In [40]:
np.random.rand(1, 1)

array([[0.32535026]])

In [8]:
np.Array

AttributeError: module 'numpy' has no attribute 'Array'